# Objetivo

Este caderno apresenta um exemplo de uso da API do ChatGPT para classificar uma série de objetos de licitação, considernado uma lista de possíveis temas. 

Ao contrário de outras abordagens, não executaremos rotinas de aprendizado supervisionado para treinar um modelo específico para cada tema. Em vez disso, o próprio chatGPT irá utilizar a sua capacidade semântica para associar cada objeto de licitação a um ou mais temas. Para melhorar a classificação, os temas podem ser melhor definidos de acordo com o interesse de cada um.

## Dependências

In [1]:
#!pip install openai pandas

## Dados de entrada

In [2]:
# Leitura da planilha contendo os objetos de licitação publicados em um dia no Comprasnet
import pandas
import numpy

df = pandas.read_excel('edital_validacao.xlsx')
objetos = df['Objeto da Contratação']

In [3]:
# Lista de objetos de licitação
objetos

0      Pregão Eletrônico -  Serviços continuados de e...
1      Pregão Eletrônico -  Contratação empresa para ...
2      Pregão Eletrônico -  Contratação de serviços d...
3      Pregão Eletrônico -  Contratação de serviços c...
4      Pregão Eletrônico -  Contratação de empresa es...
                             ...                        
152    Pregão Eletrônico -  O objeto da presente lici...
153    Pregão Eletrônico -  Cessão de uso para explor...
154    Pregão Eletrônico -  Contratação de empresa es...
155    Pregão Eletrônico -  Escolha da proposta mais ...
156    Pregão Eletrônico -  Contratação de empresa pa...
Name: Objeto da Contratação, Length: 157, dtype: object

## Lista de temas

In [4]:
lista_temas = [
    'contratação de materiais de informática, comuns e de baixa complexidade, tais como impressoras, monitores, estação de trabalho, mouse, toner, entre outros',
    'contratação de bens ou serviços de tecnologia da informação, de maior complexidade, tais como equipamentos de rede, enlaces de comunicação, datacenters, storage, softwares, servidores, cabeamentos estruturado, etc. para este tema, não considere equipamentos e serviços exclusivamente relacionados com engenharia elétrica ou de áudio-vídeo.',
    'construção, reforma, ampliação ou demolição de obras públicas',
    'obras de manutenção e conservação de rodovias, estradas ou vias públicas',
    'aquisição de bens que se caracterizem pelo requinte ou superfluidade, tais como bebidas alcólicas e alimentos caros',
    'serviços de manutenção predial, envolvendo reparos, reformas ou conservação em edificações',
    'serviços de apoio administrativo, tais como serviço de limpeza e conservação, copeiragem, recepção, segurança, cerimonial, entre outros de natureza continuada. para este tema, não considere compras exclusivas de materiais e equipamentos.',
    'serviços de transporte de pessoas, cargas ou mercadorias',
    'aquisição de veículos, frotas ou viaturas, novos ou usados, para uso do órgão ou entidade pública',
    'contratação de seguros, incluindo apólices e coberturas para proteção de bens ou pessoas pertencentes ao órgão ou entidade pública',
    'materiais de expediente e materiais de escritório, comuns e de baixa complexidade, tais como papel, grampeadores, canetas, carimbos, cola, envelopes, sabonete, papel higiênico, descartáveis, entre outros',
    'medicamentos, remédios, vacinas e próteses',
    'outros bens para hospitais e unidades de saúde, tais como insumos, implantes, equipamento de proteção individual, equipamentos médicos',
    'gêneros alimentício',
    'contratação de bens e serviços de maior complexidade técnica (ex.: equipamentos de laboratórios químicos, físicos, biológicos entre outros), que não se relacione com outros temas desta lista',
]

# Cada um dos temas será associado a uma letra maiúscula, começando pela letra 'A'.
temas = {chr(ord('A')+i):tema for i, tema in enumerate(lista_temas)}

# A letra 'ZZ' indica a falta de um tema específico para aquele objeto
temas['Z'] = 'não foi possível associar o objeto de licitação a nenhum dos temas apresentados'

# Os temas serão convertidos para um formato que será lido pelo ChatGPT
temas_str = ";\n".join([f"§{k}) {v}" for k,v in temas.items()]) + "."
print(temas_str)

§A) contratação de materiais de informática, comuns e de baixa complexidade, tais como impressoras, monitores, estação de trabalho, mouse, toner, entre outros;
§B) contratação de bens ou serviços de tecnologia da informação, de maior complexidade, tais como equipamentos de rede, enlaces de comunicação, datacenters, storage, softwares, servidores, cabeamentos estruturado, etc. para este tema, não considere equipamentos e serviços exclusivamente relacionados com engenharia elétrica ou de áudio-vídeo.;
§C) construção, reforma, ampliação ou demolição de obras públicas;
§D) obras de manutenção e conservação de rodovias, estradas ou vias públicas;
§E) aquisição de bens que se caracterizem pelo requinte ou superfluidade, tais como bebidas alcólicas e alimentos caros;
§F) serviços de manutenção predial, envolvendo reparos, reformas ou conservação em edificações;
§G) serviços de apoio administrativo, tais como serviço de limpeza e conservação, copeiragem, recepção, segurança, cerimonial, entre 

## Prompt do ChatGPT

Aqui teremos as instruções feitas ao ChatGPT para realizar a tarefa de classificação conforme desejamos.

In [5]:
instrucao_inicial = f"""
Logo a seguir, irei informar vários objetos de licitação, um em cada mensagem.
Cada objeto de licitação é identificado com um código, informado no início de cada mensagem, no formato "OBJ###", onde # indica um dígito. 
Classifique cada objeto de licitação apresentado de acordo com os temas apresentados abaixo, onde cada tema é identificado com um código no formato "§?", onde ? indica uma letras maiúscula:

{temas_str}
"""

In [6]:
instrucao_final = f"""
Informe agora todas as respostas em uma única mensagem no seguinte formato:
Cada linha deve apresentar o identificador do objeto, seguido do caractere ':', seguido dos identificadores dos temas mais relacionados a este objeto.
Não utilize espaços em branco dentro de cada linha, nem use espaços após o caractere ':'.
O identificador do tema deve ser mantido no memso formato "§?", onde ? indica uma letras maiúscula.
O identificador do objeto deve ser mantido no mesmo formato "OBJ###", onde # indica um dígito.
Os temas relacionados devem ser apresentados pelos seus identificadores, concatenados, sem espaço entre eles.
Somente apresente temas com altíssima probabilidade de estar relacionado com cada objeto.
Se uma licitação puder ser enquadrada em mais de um tema, por favor, escolha no máximo três classificações em ordem decrescente de probabilidade. 
Apresente somente as letras associadas com os temas relacionados (exemplo de linha na resposta: "OBJ005:§A,§C,§G"). 
Se não houver nenhum tema relacionado ao objeto, escolha o tema associado com a letra ZZ.
Por favor, seja conciso e utilize somente o formato solicitado. Não inclua justificativas adicionais na resposta.
"""

## Chamada à API

In [7]:
import openai

# Configuração da chave de acesso para a API
openai.api_key_path = '../secret.key'

In [8]:
# retorna as mensagems que serão utilizadas no prompt do chatGPT.
def compor_mensagens(objetos_de_licitacao):
    objetos_selecionados = [
        {"role": "user", "content": f'OBJ{i:03d} {descricao}'}
        for i, descricao in objetos_de_licitacao.items()
    ]
    messagens = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": instrucao_inicial},
            {"role": "assistant", "content": "Ok. Estou pronto."}
        ] + objetos_selecionados + [
            {"role": "user", "content": instrucao_final},
        ]    
    return messagens

In [9]:
# processa a resposta do chatGPT com a classificação dos objetos
# e retorna um dicionário contendo os temas encontrados para cada objeto 
import re
def processa_resposta(resposta):
    content = resposta['choices'][0]['message']['content']
    #print(content)
    return {
        int(i):re.findall('§([A-Z])', temas)
        for i, temas in re.findall('OBJ(\d+)[\s\.-:\)]*((?:§[A-Z],?)+)', content)
    }

In [10]:
# envia o pedido de classificação ao chatGPT e retorna
import time
def classificar_chatGPT(objetos_de_licitacao):
    messagens = compor_mensagens(objetos_de_licitacao)
    resposta = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        temperature=0.5,
        messages=messagens,
    )
    time.sleep(10)
    return processa_resposta(resposta)

In [11]:
# obtemos as classificações do chatGPT, por meio de várias requisições e agrupando as licitações 
tamanho_do_grupo = 10  # obs.: o agrupamento evita estourar o limite máximo de tokens em uma requisição
respostas = {}
for grupo_de_objetos in numpy.array_split(objetos, len(objetos)/tamanho_do_grupo):
    resposta = classificar_chatGPT(grupo_de_objetos)
    respostas.update(resposta)
    print(resposta)

{0: ['F', 'G', 'O'], 1: ['D'], 2: ['C', 'O'], 3: ['A', 'K'], 4: ['G', 'I'], 5: ['A', 'B', 'G'], 6: ['L'], 7: ['M'], 8: ['B', 'O'], 9: ['B', 'O'], 10: ['C', 'F', 'O']}
{11: ['L'], 12: ['A', 'B'], 13: ['D'], 14: ['L'], 15: ['F'], 16: ['F'], 17: ['C', 'F'], 18: ['C', 'F'], 19: ['G'], 20: ['B'], 21: ['G', 'F']}
{22: ['B'], 23: ['F'], 24: ['Z'], 25: ['Z'], 26: ['G'], 27: ['F', 'G'], 28: ['C', 'O'], 29: ['F'], 30: ['M'], 31: ['D'], 32: ['M', 'G']}
{33: ['M'], 34: ['B', 'G'], 35: ['C'], 36: ['C', 'F'], 37: ['G'], 38: ['C'], 39: ['L'], 40: ['L'], 41: ['M'], 42: ['O'], 43: ['O']}
{44: ['N'], 45: ['G'], 46: ['C'], 47: ['O'], 48: ['K'], 49: ['O', 'F'], 50: ['H', 'O'], 51: ['H', 'O'], 52: ['M'], 53: ['L'], 54: ['O']}
{55: ['C', 'O'], 56: ['F'], 57: ['G'], 58: ['O'], 59: ['G'], 60: ['C', 'F'], 61: ['H', 'G'], 62: ['K'], 63: ['H'], 64: ['C'], 65: ['O']}
{66: ['G'], 67: ['N', 'K'], 68: ['C', 'F'], 69: ['L'], 70: ['Z'], 71: ['B'], 72: ['G'], 73: ['H'], 74: ['Z'], 75: ['Z'], 76: ['Z']}
{77: ['B', 'O'],

## Saída dos dados

In [12]:
classificacao = pandas.Series(data=respostas.values(), index=respostas.keys(), name='Temas')

In [13]:
df_classificacao = pandas.concat([df, classificacao.apply(lambda x: "".join(x))], axis=1)
df_classificacao.to_csv('classificacao.csv')

## Amostragem dos resultados

In [22]:
import random
for i, classes in sorted(random.sample(list(respostas.items()), 30)):
    print(f'OBJ{i:03d}\t' + '\x1b[1;31m' + objetos.iloc[i] +'\x1b[0m')
    for c in classes:
        print('\t\x1b[1;32m' + c +'\x1b[0m' + '\t' + temas[c])
    print('\n')

OBJ010	Pregão Eletrônico -  Contratação de serviços comuns de engenharia de manutenção, adaptação, reparação e conservação de bens imóveis, com fornecimento de todos os insumos e mão de obra necessários à execução, sob a forma de Registro de Preços.
	C	construção, reforma, ampliação ou demolição de obras públicas
	F	serviços de manutenção predial, envolvendo reparos, reformas ou conservação em edificações
	O	contratação de bens e serviços de maior complexidade técnica (ex.: equipamentos de laboratórios químicos, físicos, biológicos entre outros), que não se relacione com outros temas desta lista


OBJ012	Pregão Eletrônico -  ReGISTRO DE PREÇOS PARA AQUISIÇÃO DE  EQUIPAMENTOS DE INFORMÁTICA (COMPUTADORES DESKTOP, NOTEBOOK,  TABLET, PROJETOR MULTIMÍDIA, ETC.) DESTINADOS A DIVERSOS SETORES DA  UFSM,
	A	contratação de materiais de informática, comuns e de baixa complexidade, tais como impressoras, monitores, estação de trabalho, mouse, toner, entre outros
	B	contratação de bens ou serviços

## Listagem de todos os resultados

In [14]:
for i, classes in respostas.items():
    print(f'OBJ{i:03d}\t' + '\x1b[1;31m' + objetos.iloc[i] +'\x1b[0m')
    for c in classes:
        print('\t\x1b[1;32m' + c +'\x1b[0m' + '\t' + temas[c])
    print('\n')

OBJ000	Pregão Eletrônico -  Serviços continuados de engenharia de manutenção predial, subestações, geradores, nobreaks, limpeza de reservatório de água, desobstrução de galerias, automação, controle de acesso, bombas d água, sistema de combate a incêndio, incluindo o fornecimento de mão-de-obra, de materiais, de insumos, de ferramentas, de maquinários, de EPI´s, EPC s de serviços eventuais e por demanda e todos os demais itens necessários para o atendimento às demandas de manutenção preventiva
	F	serviços de manutenção predial, envolvendo reparos, reformas ou conservação em edificações
	G	serviços de apoio administrativo, tais como serviço de limpeza e conservação, copeiragem, recepção, segurança, cerimonial, entre outros de natureza continuada. para este tema, não considere compras exclusivas de materiais e equipamentos.
	O	contratação de bens e serviços de maior complexidade técnica (ex.: equipamentos de laboratórios químicos, físicos, biológicos entre outros), que não se relacione c